# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

## Лабораторная работа 12

In [20]:
import dask.bag as db
import json
import re
import pandas as pd
import os
import dask.array as da

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [2]:
data_path = 'C:/Users/nabr9/OneDrive/Documents/reviews_full'
b = db.read_text(os.path.join(data_path, 'reviews_*.json')).map(json.loads)
b.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

In [7]:
b

dask.bag<loads, npartitions=6>

In [19]:
type(db.read_text(os.path.join(data_path, 'reviews_*.json')).take(1))

tuple

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [8]:
def rating(a):
    rate = int(re.sub('.json', '', re.sub('C:/Users/nabr9/OneDrive/Documents/reviews_full/reviews_', '', a[1])))
    a = json.loads(a[0])
    a.update({'rating': rate})
    return a

In [9]:
b1 = db.read_text(os.path.join(data_path, 'reviews_*.json'), include_path = True).map(rating)
b1.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

3. Посчитайте количество отзывов в исходном датасете.

In [10]:
b1.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [11]:
b11 = b1.filter(lambda x: x['date'][:4] == '2014' or x['date'][:4] == '2015')
b11.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gallo, s

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [12]:
def processing(a):
    a['review'] = re.sub(r"[^a-zA-Z\s]+", '', a['review'].lower().lstrip().rstrip())
    return a

In [13]:
b12 = b11.map(processing)

In [14]:
b12.take(10)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'simple and easy way to enjoy a slice of pizza any time  welltoasted bread is the key  really toast it  i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor  i used sourdough bread  medium cheddar cheese  fast  fun  great idea  made for  hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'delish  i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans  i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic  added the olives under the cheese and baked uncovered for the  minutes  served with pico de gallo sour cream and avocado chunks  fantastic  tha

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [15]:
b12.count().compute()

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка

In [16]:
list(b12.pluck('rating').frequencies())

[(0, 42472), (1, 9246), (2, 9380), (3, 26532), (4, 119413), (5, 528231)]

8. Найдите среднее значение `rating` в выборке

In [11]:
c = b12.pluck('rating').mean().compute(); c

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.